# Imports

In [1]:
import pandas as pd, numpy as np

---

# Carga de datos

In [2]:
auctions = pd.read_pickle("../../data/tp2/auctions_tp2_formateado.pkl")

In [3]:
installs = pd.read_pickle("../../data/tp2/installs_tp2_formateado.pkl")

In [4]:
events = pd.read_pickle("../../data/tp2/events_tp2_formateado.pkl")

In [5]:
clicks = pd.read_pickle("../../data/tp2/clicks_tp2_formateado.pkl")

---

# Cargo targets de la competencia

In [6]:
targets = pd.read_pickle("../../data/tp2/ref_hashes_target.pkl")

---

# Renombro columnas

In [8]:
events.rename(columns={"ref_hash" : "device_id"}, inplace=True)

In [9]:
installs.rename(columns={"ref_hash" : "device_id"}, inplace=True)

In [10]:
clicks.rename(columns={"ref_hash" : "device_id"}, inplace=True)

---

# Armo ventanas

In [11]:
for ventana_nro in range(1,6):
    auctions["ventana_{}".format(ventana_nro)] = (((17+ventana_nro) <= auctions["date"].dt.day) & ((auctions["date"].dt.day) <= (19+ventana_nro)))

In [12]:
auctions["ventana_final"] = (((25) <= auctions["date"].dt.day) & ((auctions["date"].dt.day) <= (27)))

In [13]:
for ventana_nro in range(1,6):
    events["ventana_{}".format(ventana_nro)] = (((17+ventana_nro) <= events["date"].dt.day) & ((events["date"].dt.day) <= (19+ventana_nro)))

In [14]:
events["ventana_final"] = (((25) <= events["date"].dt.day) & ((events["date"].dt.day) <= (27)))

In [15]:
for ventana_nro in range(1,6):
    clicks["ventana_{}".format(ventana_nro)] = (((17+ventana_nro) <= clicks["created"].dt.day) & ((clicks["created"].dt.day) <= (19+ventana_nro)))

In [16]:
clicks["ventana_final"] = (((25) <= clicks["created"].dt.day) & ((clicks["created"].dt.day) <= (27)))

In [17]:
for ventana_nro in range(1,6):
    installs["ventana_{}".format(ventana_nro)] = (((17+ventana_nro) <= installs["created"].dt.day) & ((installs["created"].dt.day) <= (19+ventana_nro)))

In [18]:
installs["ventana_final"] = (((25) <= installs["created"].dt.day) & ((installs["created"].dt.day) <= (27)))

---

# Armo set de datos con las presencias por ventana y dataset

In [23]:
for ventana_nro in range(1, 6):
    targets["en_v{}_auctions".format(ventana_nro)] = targets.index.isin(auctions.loc[auctions["ventana_{}".format(ventana_nro)]]["device_id"])
    targets["en_v{}_events".format(ventana_nro)] = targets.index.isin(events.loc[events["ventana_{}".format(ventana_nro)]]["device_id"])
    targets["en_v{}_clicks".format(ventana_nro)] = targets.index.isin(clicks.loc[clicks["ventana_{}".format(ventana_nro)]]["device_id"])
    targets["en_v{}_installs".format(ventana_nro)] = targets.index.isin(installs.loc[installs["ventana_{}".format(ventana_nro)]]["device_id"])

In [26]:
targets["en_v{}_auctions".format("final")] = targets.index.isin(auctions.loc[auctions["ventana_{}".format("final")]]["device_id"])
targets["en_v{}_events".format("final")] = targets.index.isin(events.loc[events["ventana_{}".format("final")]]["device_id"])
targets["en_v{}_clicks".format("final")] = targets.index.isin(clicks.loc[clicks["ventana_{}".format("final")]]["device_id"])
targets["en_v{}_installs".format("final")] = targets.index.isin(installs.loc[installs["ventana_{}".format("final")]]["device_id"])

In [31]:
targets["columna_para_conteo"] = 1

In [33]:
targets.columns.values

array(['en_v1_auctions', 'en_v2_auctions', 'en_v3_auctions',
       'en_v4_auctions', 'en_v5_auctions', 'en_v1_events', 'en_v1_clicks',
       'en_v1_installs', 'en_v2_events', 'en_v2_clicks', 'en_v2_installs',
       'en_v3_events', 'en_v3_clicks', 'en_v3_installs', 'en_v4_events',
       'en_v4_clicks', 'en_v4_installs', 'en_v5_events', 'en_v5_clicks',
       'en_v5_installs', 'en_vfinal_auctions', 'en_vfinal_events',
       'en_vfinal_clicks', 'en_vfinal_installs', 'columna_para_conteo'],
      dtype=object)

In [49]:
targets.groupby(['en_v1_auctions', 'en_v2_auctions', 'en_v3_auctions',
       'en_v4_auctions', 'en_v5_auctions', 'en_v1_events', 'en_v1_clicks',
       'en_v1_installs', 'en_v2_events', 'en_v2_clicks', 'en_v2_installs',
       'en_v3_events', 'en_v3_clicks', 'en_v3_installs', 'en_v4_events',
       'en_v4_clicks', 'en_v4_installs', 'en_v5_events', 'en_v5_clicks',
       'en_v5_installs', 'en_vfinal_auctions', 'en_vfinal_events',
       'en_vfinal_clicks', 'en_vfinal_installs']).agg("count").sort_values(by="columna_para_conteo", ascending=False)

columna_para_conteo
en_v1_auctions en_v2_auctions en_v3_auctions en_v4_auctions en_v5_auctions en_v1_events en_v1_clicks en_v1_installs en_v2_events en_v2_clicks en_v2_installs en_v3_events en_v3_clicks en_v3_installs en_v4_events en_v4_clicks en_v4_installs en_v5_events en_v5_clicks en_v5_installs en_vfinal_auctions en_vfinal_events en_vfinal_clicks en_vfinal_installs                     
True           True           True           True           True           False        False        False          False        False        False          False        False        False          False        False        False          False        False        False          True               True             False            True                                196
False          False          False          False          False          False        False        False          False        False        False          False        False        False          False        False        False          False        False        False          True               True             False            True                                132
True           True           True           True           True           False        False        False          False        False        False          False        False        False          False        False        False          False        False        False          True               False            False            True                                 67
                                                                                                                                                             True         False        True           True         False        True           True         False        True           True               False            False            False                                64
False          False          False          False          False          False        False        False          False        False        False          False        False        False          False        False        False          False        False        False          True               False            False            True                                 56
True           True           True           True           True           False        False        False          True         False        True           True         False        True           True         False        True           False        False        False          True               False            False            False                                55
                                                                           True         False        True           True         False        True           True         False        True           False        False        False          False        False        False          True               False            False            False                                49
                                                                                                                                                             False        False        False          False        False        False          False        False        False          True               False            False            False                                49
                                                                           False        False        False          False        False        False          False        False        False          False        False        False          True         False        True           True               False            False            False                                46
                                                                                                                                                                                                      True         False        True           True       